## INTELIGENCIA ARTIFICIAL 
## = LABORATORIO 4 =

Jose Miguel Gonzalez
20335

Tarea 1.1
Leer el fichero CSV proporcionado y almacenarlo en un np.array


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [ ]:
# Step 1: Load the dataset
data = pd.read_csv("framingham.csv")

# Remove the rows with missing values
data = data.dropna()
data = pd.DataFrame(data)
data = data.drop(columns=['education'])
data = data.drop_duplicates()

# Select the independent variables (features)
X = data[["age", "cigsPerDay", "prevalentHyp", "sysBP", "BMI"]].values
# Select the dependent variable (target)
y = data["TenYearCHD"].values

Tarea 1.2
Ajuste un modelo logístico polinómico basado en el conjunto de datos cargado en la matriz que relacione las variables independientes que considere apropiadas (puede no utilizar todos los componentes de X, con
variables independientes que considere oportunas (puede no utilizar todos los componentes de X), con la variable de salida dependiente (sufre o no de
variable de salida dependiente (sufre o no una parada cardiaca).

In [ ]:
# Split the data into training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42)

# Scale the data using standardization
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

# Replace NaN values with median values
imputer = SimpleImputer(strategy='median')
X_train_std = imputer.fit_transform(X_train_std)
X_test_std = imputer.transform(X_test_std)

# Fit a polynomial logistic regression model using the training data
model = Pipeline([
("poly", PolynomialFeatures(degree=2)),
("logreg", LogisticRegression(random_state=42))
])

model.fit(X_train_std, y_train)

Tarea 1.3
Use la implementación vectorial del algoritmo de regresión logística (descenso gradiente visto en clase).

In [ ]:
class regresionLogica:

    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True, verbose=False):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.verbose = verbose

    def add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

    def fit(self, X, y):
        if self.fit_intercept:
            X = self.add_intercept(X)

        # weights initialization
        self.theta = np.zeros(X.shape[1])

        for i in range(self.num_iter):
            z = np.dot(X, self.theta)
            h = self.sigmoid(z)
            gradient = np.dot(X.T, (h - y)) / y.size
            self.theta -= self.lr * gradient

            if self.verbose and i % 10000 == 0:
                z = np.dot(X, self.theta)
                h = self.sigmoid(z)
                print(f'Loss: {self.loss(h, y)}')

    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.add_intercept(X)

        return self.sigmoid(np.dot(X, self.theta))

    def predict(self, X, threshold=0.5):
        return self.predict_prob(X) >= threshold

In [ ]:
# Define and train a logistic regression model using gradient descent algorithm
lr = regresionLogica(lr=0.1, num_iter=100000)
lr.fit(X, y)

# Predict the output using the trained model
y_pred = lr.predict(X)

# Evaluate the performance of the model
accuracy = (y_pred == y).mean()

Tarea 1.4
Mediante validación cruzada, determine el grado del polinomio que mejor describa la nube de puntos (encuentre el mejor equilibrio entre la adhesión a los datos de entrenamiento y el grado del polinomio).
equilibrio entre la adherencia a los datos de entrenamiento y la generalización a datos no observados previamente).

In [ ]:
# Define the pipeline
model = Pipeline([
("poly", PolynomialFeatures()),
("logreg", LogisticRegression())
])

# Define the parameters to search over
params = {
"poly__degree": [1, 2, 3, 4, 5]
}

# Perform cross-validation
grid = GridSearchCV(model, params, cv=5)
grid.fit(X_train_std, y_train)

# Print the best parameters and score
print("Best degree:", grid.best_params_["poly__degree"])
print("Best score:", grid.best_score_)

Tarea 1.5
Realice un análisis de sus resultados.